# Query Transformations

### Imports and configs

In [1]:
from typing import List
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI
import faiss
import os
import sys
from dotenv import load_dotenv
import hashlib
import pickle


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

EMBED_DIMENSION = 512
CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=EMBED_DIMENSION)

path = "../data/"
node_parser = SimpleDirectoryReader(input_dir=path, required_exts=['.txt', '.pdf'])
documents = node_parser.load_data()
### Set up vector store retriever
class TextCleaner(TransformComponent):
    """
    Transformation to be used within the ingestion pipeline.
    Cleans clutters from texts.
    """
    def __call__(self, nodes, **kwargs) -> List[BaseNode]:
        
        for node in nodes:
            node.text = node.text.replace('\t', ' ') # Replace tabs with spaces
            node.text = node.text.replace(' \n', ' ') # Replace paragraph seperator with spacaes
            
        return nodes
CACHE_DIR = "../cache"
VECTOR_STORE_PATH = os.path.join(CACHE_DIR, "faiss_index.pkl")
HASH_PATH = os.path.join(CACHE_DIR, "documents_hash.txt")

def hash_documents(documents):
    # combine all the texts into a single string
    all_titles = [doc.metadata['file_name'] for doc in documents]
    all_titles_distinct = list(set(all_titles))
    all_titles_distinct.sort()
    all_titles_str = " ".join(all_titles_distinct)
    # return a hash of the combined text which will stay consistent if the text is the same across multiple runs
    return hashlib.md5(all_titles_str.encode('utf-8')).hexdigest()

def load_or_create_vector_store(documents, embed_dim, chunk_size, chunk_overlap):
    os.makedirs(CACHE_DIR, exist_ok=True)
    
    current_hash = hash_documents(documents)
    
    if os.path.exists(HASH_PATH) and os.path.exists(VECTOR_STORE_PATH):
        with open(HASH_PATH, 'r') as f:
            stored_hash = f.read().strip()

        if stored_hash == current_hash:
            print("Loading vector store from cache...")
            with open(VECTOR_STORE_PATH, 'rb') as f:
                return pickle.load(f)
    
    print("Creating new vector store...")
    faiss_index = faiss.IndexFlatL2(embed_dim)
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    
    text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    pipeline = IngestionPipeline(
        transformations=[
            TextCleaner(),
            text_splitter,
        ],
        vector_store=vector_store,
    )
    
    nodes = pipeline.run(documents=documents)
    vector_store_index = VectorStoreIndex(nodes)
    
    # Save the new vector store and hash
    with open(VECTOR_STORE_PATH, 'wb') as f:
        pickle.dump(vector_store_index, f)
    
    with open(HASH_PATH, 'w') as f:
        f.write(current_hash)
    
    return vector_store_index

vector_store_index = load_or_create_vector_store(documents, EMBED_DIMENSION, CHUNK_SIZE, CHUNK_OVERLAP)
retriever = vector_store_index.as_retriever(similarity_top_k=2)

/Users/user/Projects/ragbrag_pycon_ie_24/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Loading vector store from cache...


### Query transformations
#### Query rewriting

In [2]:
test_query = "What is the SNP's policy on climate change?"

query_gen_str = """\
You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system. 
Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information.
Original Query: {query}
Rewritten Query:"""
query_gen_prompt = PromptTemplate(query_gen_str)

llm = OpenAI(model="gpt-4o", temperature=0, max_tokens=4000)

def generate_query(query: str, llm, query_gen_prompt):
    response = llm.predict(
        query_gen_prompt, query=query
    )
    # assume LLM proper put each query on a newline
    new_query = response.split("\n")
    query_str = "\n".join(new_query)
    return query_str

generated_query = generate_query(test_query, llm, query_gen_prompt)
print(f"original query: {test_query}")
print(f"generated query: {generated_query}")
# Compare improved query response
query_engine = vector_store_index.as_query_engine()
response_simple = query_engine.query(test_query)
response_improved = query_engine.query(generated_query)
print(f"Simple query response: {response_simple}")
print(f"Improved query response: {response_improved}")

original query: What is the SNP's policy on climate change?
generated query: What specific measures and initiatives does the Scottish National Party propose to address climate change, and how do they plan to implement these strategies within Scotland?
Simple query response: The SNP's policy on climate change involves a commitment to tackling climate change and nature loss, emphasizing the importance of emissions reduction and economic prosperity going hand in hand. They aim to promote significant growth in renewables, storage, hydrogen, and carbon capture as the best pathway to achieving net zero emissions and ensuring secure, affordable, and clean energy. Additionally, the SNP wants to rule out new nuclear power plants in Scotland and promote Scotland's hydrogen export potential, pressing for progress in direct interconnection between Scotland and the continent to unlock Scotland's renewable potential.
Improved query response: The Scottish National Party proposes to establish a Four N

#### Step-back Prompting: Generating broader queries for better context retrieval.

In [3]:
test_query = "What is the SNP's policy on carbon emissions?"
step_back_template = """You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.

Original query: {query}

Step-back query:"""
query_gen_prompt = PromptTemplate(step_back_template)
generated_query = generate_query(test_query, llm, query_gen_prompt)
print(f"original query: {test_query}")
print(f"generated query: {generated_query}")

original query: What is the SNP's policy on carbon emissions?
generated query: What are the general policies and positions of the SNP on environmental issues?


#### Sub-query decomposition

In [4]:
test_query = "What is the Conservative Party's stance on immigration?"

subquery_decomposition_template = """You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.

Original query: {query}

example: What are the impacts of climate change on the environment?

Sub-queries:
1. What are the impacts of climate change on biodiversity?
2. How does climate change affect the oceans?
3. What are the effects of climate change on agriculture?
4. What are the impacts of climate change on human health?"""

query_gen_prompt = PromptTemplate(subquery_decomposition_template)
generated_query = generate_query(test_query, llm, query_gen_prompt)
print(f"original query: {test_query}")
print(f"generated query: {generated_query}")

original query: What is the Conservative Party's stance on immigration?
generated query: Sub-queries:
1. What are the key policies of the Conservative Party regarding immigration?
2. How has the Conservative Party's stance on immigration changed over recent years?
3. What are the main arguments the Conservative Party uses to support its immigration policies?
4. How do the Conservative Party's immigration policies compare to those of other major political parties?
